In [ ]:
import pandas as pd

# 1. 데이터 불러오기
# 데이터 불러오기 (DataFrame 객체로 로드)
train_path = '/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/train.csv'
train_df = pd.read_csv(train_path) # train 데이터를 DataFrame으로 로드
test_path = '/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/test.csv'
test_df = pd.read_csv(test_path)

# 2. EDA
# 보고서 생성 (target 인자 없이)
# from dataprep.eda import create_report # dataprep 데이터 분석
# report = create_report(train_df) # 여기서 target 인자를 삭제했습니다.
# 보고서를 HTML 파일로 저장
# 파일명은 train_dataprep_report.html로 지정합니다.
# report.save('train_dataprep_report.html')

# 데이터 분석
# 1.keyword 칼럼:
# Approximate Distinct Count: 221개
# Approximate Unique (%): 2.9%
# Missing: 61개
# Missing (%): 0.8%
# Memory Size: 556863
# 인사이트: sweetviz와 유사하게 221개의 고유 키워드가 있고, 약 0.8%의 적은 결측치가 존재합니다. 이 칼럼은 충분히 활용 가치가 있어 보입니다. 결측치는 처리해주는 것이 좋습니다.
# 2. location 칼럼:
# Approximate Distinct Count: 3341개
# Approximate Unique (%): 65.8%
# Missing: 2533개
# Missing (%): 33.3%
# Memory Size: 404598
# 인사이트: sweetviz에서 확인했던 것과 동일하게, 3341개의 매우 많은 고유 지역이 있으며, 무려 33.3%라는 높은 비율의 결측치가 존재합니다.


# ========================================
# 텍스트 전처리(정제) 함수 정의 섹션
# ========================================
# 토큰화 필요 함수
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# NLTK 데이터 다운로드 확인
try:
    nltk.data.find("tokenizers/punkt")
    nltk.data.find("corpora/stopwords")
except nltk.downloader.DownloadError:
    nltk.download("punkt")
    # nltk.download('punkt_tab')
    nltk.download("stopwords")
def preprocess_text(df, keyword_mode=None):
    """
    텍스트 전처리를 수행하는 함수
    - keyword 결측치 처리
    - 소문자 변환
    - 특수문자/숫자 제거
    - 토큰화
    - 불용어 제거

    Parameters:
    -----------
    df : DataFrame
        전처리할 데이터프레임
    keyword_mode : str, optional
        keyword 컬럼의 최빈값. None이면 df에서 자동 계산

    Returns:
    --------
    df : DataFrame
        전처리가 완료된 데이터프레임
    keyword_mode : str
        사용된 keyword 최빈값 (test 데이터에 재사용하기 위해 반환)
    """
    df = df.copy()  # 원본 데이터 보존
    # 3. 텍스트 정제
    # 3.1. keyword 결측치를 최빈값으로 채우기
    if keyword_mode is None:
        keyword_mode = df['keyword'].mode()[0]
    df['keyword'] = df['keyword'].fillna(keyword_mode)
    # 3.2. 대문자를 소문자로 변환
    df['text'] = df['text'].str.lower()
    # 3.3. 특수문자, 숫자, 기호 제거 (a-z와 공백만 남김)
    df['text'] = df['text'].str.replace(r'[^a-z ]', '', regex=True)
    # 4. 토큰화: 문장을 단어 단위로 분리
    df['tokenized_text'] = df['text'].apply(word_tokenize)
    # 5. 불용어 제거: 의미 없는 단어(is, a, the 등) 제거
    stop_words = set(stopwords.words('english'))
    df['non_stopwords_text'] = df['tokenized_text'].apply(
        lambda tokens: [word for word in tokens if word not in stop_words]
    )
    return df, keyword_mode

# 3. 텍스트 정제 (전처리 함수 사용)
train_df, keyword_mode = preprocess_text(train_df)
print("전처리 완료!")
print("컬럼 확인:", "non_stopwords_text" in train_df.columns)
print("전처리 후 결측치 확인:")
print(train_df.isnull().sum())

# 6. 벡터화 (Word2Vec 모델 학습)
# Word2Vec은 단어를 벡터(숫자 배열)로 변환하여, 비슷한 의미를 가진 단어들이
# 벡터 공간에서 가까이 위치하도록 학습하는 알고리즘입니다.
from gensim.models import Word2Vec
# 6.1. 학습 데이터 준비
# non_stopwords_text 컬럼에는 불용어가 제거된 토큰 리스트가 들어있습니다.
# 예: [['fire', 'building'], ['disaster', 'earthquake'], ...]
sentences = train_df["non_stopwords_text"].tolist()
print(f"Word2Vec 학습 문장 개수: {len(sentences)}개")
# 6.2. Word2Vec 모델 학습
# 하이퍼파라미터 설정:
# - vector_size=100: 각 단어를 100차원의 벡터로 표현
# - window=5: 현재 단어를 기준으로 앞뒤 5개 단어까지 문맥으로 고려
# - min_count=3: 3번 미만 등장한 단어는 학습에서 제외 (노이즈 제거)
# - workers=4: 병렬 처리를 위한 CPU 코어 수
embedding_model = Word2Vec(sentences, vector_size=100, window=5, min_count=3, workers=4)
print(f"학습된 단어 개수: {len(embedding_model.wv.index_to_key)}개")
# 6.3. 단어 사전 생성
# 단어를 고유한 정수 인덱스로 매핑하는 사전을 만듭니다.
# 예: {'fire': 0, 'disaster': 1, 'building': 2, ...}
# 이 사전은 이후 정수 인코딩 단계에서 사용됩니다.
word_to_index = {word: idx for idx, word in enumerate(embedding_model.wv.index_to_key)}
print(f"단어 사전 크기: {len(word_to_index)}개")
# 6.4. Word2Vec 가중치 추출
# Word2Vec이 학습한 단어 벡터들을 PyTorch 텐서로 변환합니다.
# 이 가중치는 나중에 LSTM 모델의 Embedding 층 초기값으로 사용됩니다.
# pretrained_weights 형태: [vocab_size, vector_size] = [단어개수, 100]
import torch
pretrained_weights = torch.FloatTensor(embedding_model.wv.vectors)
print(f"임베딩 가중치 형태: {pretrained_weights.shape}")
# 6.5. 학습 결과 확인 (선택사항)
# 'disaster'와 의미가 유사한 단어 5개를 찾아봅니다.
# 벡터 공간에서 가까운 단어들을 찾아 Word2Vec이 잘 학습되었는지 확인합니다.
try:
    similar_words = embedding_model.wv.most_similar("disaster", topn=5)
    print("\n'disaster'와 유사한 단어:")
    for word, similarity in similar_words:
        print(f"  - {word}: {similarity:.4f}")
except KeyError:
    print("'disaster' 단어가 단어 사전에 없습니다.")


# 7. 정수 인코딩 & 패딩
def encode_and_pad(df, word_to_index, max_len=None, has_target=True):
    """
    정수 인코딩 및 패딩 수행

    Why?
    - 정수 인코딩과 패딩은 항상 함께 수행되므로 하나의 함수로 통합
    - train/test 데이터에 동일한 전처리를 보장

    Parameters:
    -----------
    df : DataFrame
        처리할 데이터프레임 (non_stopwords_text 컬럼 필요)
    word_to_index : dict
        단어 사전 (Word2Vec 학습 시 생성됨)
    max_len : int, optional
        패딩 길이. None이면 df에서 최대 길이 계산
    has_target : bool
        target 컬럼 포함 여부 (train: True, test: False)

    Returns:
    --------
    X : torch.Tensor
        패딩된 입력 텐서 [샘플 수, max_len]
    y : torch.Tensor or None
        타겟 텐서 (has_target=False면 None)
    max_len : int
        사용된 최대 길이 (test 데이터 처리 시 재사용)
    """
    # 7.1. 정수 인코딩: 단어를 숫자로 변환
    df["encoded_text"] = df["non_stopwords_text"].apply(
        lambda tokens: [
            word_to_index[token] for token in tokens if token in word_to_index
        ]
    )
    # 7.2. 최대 길이 계산 (train에서만)
    if max_len is None:
        max_len = max(len(s) for s in df["encoded_text"])

    # 7.3. 패딩: 모든 시퀀스를 max_len 길이로 맞춤
    padded_sequences = [seq + [0] * (max_len - len(seq)) for seq in df["encoded_text"]]

    # 텐서 변환
    X = torch.LongTensor(np.array(padded_sequences))

    # target이 있으면 y도 반환
    y = None
    if has_target:
        y = torch.LongTensor(df["target"].values)

    return X, y, max_len


# 8. 데이터 분할
# Word2Vec 기반 정수 인코딩 및 패딩된 시퀀스와 목표 변수 'target'을 학습용, 검증용으로 나눕니다.
# test_size는 검증용 데이터의 비율을 의미합니다. (0.2는 20%)
# random_state는 재현성을 위해 고정합니다.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)
# 데이터 로더 구축 전에 텐서를 디바이스로 이동
# GPU 사용 설정 (맥북은 'mps')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")
# 텐서를 디바이스로 이동시키는 코드 추가
X_train = X_train.to(device)
X_val = X_val.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)

# 9.데이터 로더 구축
# Dataset: 데이터셋의 **데이터(X)**와 **정답 레이블(Y)**을 묶어주는 역할을 합니다. 각 데이터 샘플에 쉽게 접근할 수 있도록 만들어 주죠.
# DataLoader: Dataset을 감싸서 미니 배치(mini-batch) 단위로 데이터를 자동으로 묶어주고, 학습 과정에서 데이터를 모델에 전달하는 역할을 합니다. 이렇게 하면 전체 데이터를 한꺼번에 메모리에 올리지 않고도 효율적으로 학습할 수 있어요.
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        # 데이터셋의 전체 길이를 반환
        return len(self.X)
    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터와 레이블을 반환
        return self.X[idx], self.y[idx]
# 데이터셋 객체 생성
train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)
# DataLoader 객체 생성
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
print(f"학습 데이터로더 배치 수: {len(train_loader)}")
print(f"검증 데이터로더 배치 수: {len(val_loader)}")
# 첫 번째 배치 확인
for X_batch, y_batch in train_loader:
    print(f"첫 번째 배치 데이터 형태: {X_batch.shape}")
    print(f"첫 번째 배치 레이블 형태: {y_batch.shape}")
    break

# 10. 딥러닝 모델 구축
import torch.nn as nn # PyTorch의 신경망 모듈(패키지)
class LSTMClassifier(nn.Module): # nn.Module을 상속받아 LSTMClassifier 클래스 정의
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, pretrained_weights=None):
        # vocab_size: 단어 사전의 크기. 모델이 알아야 할 단어의 총 개수
        # embedding_dim: 각 단어를 표현하는 임베딩 벡터의 차원, 각 단어를 몇 차원의 벡터로 표현할지를 결정.
        # hidden_dim: LSTM의 은닉 상태(hidden state) 차원. LSTM 층이 학습하는 정보의 '깊이'를 결정.
        # output_dim: 모델의 출력 차원. 분류 문제에서는 클래스의 수와 동일. 지금은 이진 분류이므로 0(재난 트윗 아님) 또는 1(재난 트윗)을 나타낼 수 있도록 보통 1로 설정.
        # num_layers: LSTM 층을 설정. 기본 1개.
        super().__init__() # 부모 클래스 초기화
        # 레이어(층) 정의
        # 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) # 패딩 인덱스 0으로 설정. 패딩으로 설정된 0을 학습에서 제외시키기 위해서 설정.
        # Word2Vec 가중치가 있으면 로드
        if pretrained_weights is not None:
            self.embedding.weight.data.copy_(pretrained_weights)
            # Fine-tuning 허용 (학습 중 임베딩도 업데이트 됨)
            self.embedding.weight.requires_grad = True
        # LSTM 레이어
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True) # num_layers 추가: 층수만큼 스택
        # 완전연결 레이어
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, text):
        # text = [batch size, seq len]
        embedded = self.embedding(text)
        # 입력 데이터 text를 임베딩 층에 통과시켜 단어 벡터로 변환합니다.  이 결과로 [batch size, seq len, embedding dim] 형태의 3차원 텐서가 생성되는데, 이것은 "여러 문장(batch size)에 있는 각 단어(seq len)를 벡터(embedding dim)로 표현했다"는 뜻입니다.
        # embedded = [batch size, seq len, embedding dim]
        output, (hidden, cell) = self.lstm(embedded)
        # 임베딩된 텐서가 LSTM 층을 통과합니다. 이때 LSTM은 두 가지를 출력합니다.
        # output: 각 시점(time step)의 출력.
        #hidden: 마지막 시점의 은닉 상태(Hidden State).
        # cell: 마지막 시점의 셀 상태(Cell State).
        # output = [batch size, seq len, hidden dim]
        # hidden = [1, batch size, hidden dim]
        # hidden = hidden.squeeze(0)
        # hidden 텐서의 형태가 [1, batch size, hidden dim]인데, LSTM은 기본적으로 하나의 층을 사용하므로 맨 앞의 차원(1)은 필요가 없습니다. squeeze(0)를 사용해 불필요한 차원을 제거하여 [batch size, hidden dim] 형태로 만들어줍니다.
        # hidden = [batch size, hidden dim]
        hidden = hidden[-1]  # [num_layers, batch_size, hidden_dim]의 마지막 층 추출: [batch_size, hidden_dim]
        return self.fc(hidden)
        # 마지막으로, 이렇게 정리된 hidden 텐서를 완전 연결 층(fc)에 넣어 최종 분류 결과를 얻습니다. 이때 hidden은 "문장 전체의 의미를 압축한 벡터"라고 생각할 수 있습니다.
import torch.optim as optim
# 하이퍼파라미터 정의
vocab_size = len(word_to_index) # 단어 사전 크기
embedding_dim = 100 # vector_size
hidden_dim = 512 # LSTM의 은닉 상태 크기(모델이 정보를 기억하여 복잡한 패턴을 학습)
num_layers = 2 # LSTM 층 개수를 설정. 기본 1, 증가 시 모델 깊이 증가로 복잡한 시퀀스 패턴 학습 강화)
output_dim = 1 # 최종 출력 차원, 이진 분류이므로 1로 설정
# Word2Vec 가중치를 PyTorch 텐서로 변환
pretrained_weights = torch.FloatTensor(embedding_model.wv.vectors)
# 모델 객체 생성
model = LSTMClassifier(vocab_size
                     , embedding_dim
                     , hidden_dim
                     , output_dim
                     , num_layers
                     , pretrained_weights=pretrained_weights # 가중치 적용
                       )
# 손실 함수 정의: BCEWithLogitsLoss(이진 교차 엔트로피 손실 함수)
criterion = nn.BCEWithLogitsLoss()
# 최적화 정의: Adam(최적화 알고리즘)
optimizer = optim.Adam(model.parameters())

# 11. 모델 학습 및 평가 (훈련 루프)
# 학습 루프(Training Loop)
# 모델이 데이터로부터 지식을 습득하는 과정입니다. 데이터를 미니 배치 단위로 모델에 주입하고, 모델의 예측과 실제 정답 사이의 차이(손실)를 계산합니다. 이 손실을 줄이기 위해 역전파(Backpropagation)를 통해 모델의 가중치들을 조금씩 조정합니다.
# 평가 루프(Validation Loop)
# 모델이 학습 데이터에만 과적합되지 않았는지 확인하는 과정입니다. 학습이 끝날 때마다 별도로 분리해 둔 검증 데이터셋으로 모델의 성능을 평가합니다. 이때는 가중치를 업데이트하지 않고 오직 예측 정확도만 계산합니다.
def train_model(model, train_loader, optimizer, criterion):
    # 모델 훈련의 핵심 부분으로, 미니 배치 단위로 학습.
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for inputs, labels in train_loader:
        # 1. 그래디언트 초기화: 이전 배치의 그래디언트를 초기화
        # 그래디언트는 기울기또는 변화율을 말한다.
        optimizer.zero_grad()
        # 2. 예측값 계산: 입력 데이터를 모델에 넣어 계산
        outputs = model(inputs)
        # 3. 손실 계산: 예측값과 정답을 비교하여 손실을 계산
        loss = criterion(outputs.squeeze(1), labels.float()) # 텐서의 형태를 위해 형태 변환
        # 4. 역전파: 계산된 손실을 사용하여 모델의 모든 학습 가능한 매개변수에 대한 그래디언트를 계산. 손실을 최소화 하는 방향 제시. 역전파가 그래디언트를 계산하는 알고리즘이고 계산된 그래디언트를 사용하여 가중치를 업데이트하는 최적화 알고리즘을 경사하강법이라고 한다.
        loss.backward()
        # 5. 가중치 업데이트: 옵티마이저는 역전파를 통해 계산된 그래디언트 값을 사용하여 모델의 모든 가중치를 업데이트.
        optimizer.step()
        # 예를 들어, 3번 채점하여 4번에서 어디를 어떻게 공부할지 분석 5번 실제로 공부하여 실력 향상
        # 훈련 진행 상황 추적(1~5까지 한 에포크)
        total_loss += loss.item()
        preds = torch.round(torch.sigmoid(outputs)) # FC 층의 출력은 제한이 없다. 음수, 양수 등 아무거나 나올 수 있는데 우리는 0 or 1을 원하기 때문에 sigmoid 함수를 적용(sigmoid는 확률로 해석이 가능)시켜 0~1 사이 값으로 표현되게 변경하여 반올림해서 0 또는 1로 만들어 준다.
        total_correct += (preds.squeeze(1) == labels).sum().item()
        total_samples += labels.size(0)
    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

def evaluate_model(model, val_loader, criterion):
    # 평가 함수
    model.eval() # 모델을 평가모드로 전환. Dropout이나 BatchNorm 같은 일부 층들이 비활성화
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        # 학습이 아닌 결과 값을 확인해야하니 그래디언트 계산을 비활성화하여 가중치가 변경되지 않도록 함.
        for inputs, labels in val_loader:
            # 1. 예측값 계산
            outputs = model(inputs)
            # 2. 손실 계산
            loss = criterion(outputs.squeeze(1), labels.float())
            # 훈련 진행 상황 추적
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs))
            total_correct += (preds.squeeze(1) == labels).sum().item()
            total_samples += labels.size(0)
    avg_loss = total_loss / len(val_loader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

# 에포크 수 설정: 모델을 5회 반복(10회 반복 시, 과적합 진행)
N_EPOCHS = 5
# 학습 및 평가 결과 저장을 위한 리스트
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
# GPU 사용 설정 (맥이기에 'mps')
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

# 전체 학습 루프: 핵심 반복문
for epoch in range(N_EPOCHS):
    # 훈련
    train_loss, train_acc = train_model(model, train_loader, optimizer, criterion)
    # 검증
    val_loss, val_acc = evaluate_model(model, val_loader, criterion)
    # 결과 저장
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    # 결과 출력
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal. Loss: {val_loss:.3f} | Val. Acc: {val_acc*100:.2f}%')

# ------------------------
# 테스트 데이터
# ------------------------

# 1. 테스트 데이터 불러오기
test_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/test.csv'
test_df = pd.read_csv(test_path)
print("테스트 데이터 로드 완료.")

# 2. 테스트 데이터 전처리 (훈련 데이터와 동일한 방식 적용)
# train 데이터와 동일한 전처리를 수행하되, keyword_mode는 train에서 사용한 값을 그대로 사용
# 이렇게 하면 train과 test의 전처리 방식이 완전히 동일해집니다.
test_df, _ = preprocess_text(test_df, keyword_mode=keyword_mode)
print("테스트 데이터 전처리 완료 (train과 동일한 방식 적용)")

# 3. 정수 인코딩 및 패딩 (고도화 함수 사용)
# encode_and_pad 함수를 사용하여 Train과 동일한 방식으로 처리
X_test, _, _ = encode_and_pad(
    test_df, 
    word_to_index, 
    max_len=max_len,      # Train에서 계산한 최대 길이 재사용
    has_target=False       # Test 데이터에는 target 없음
)
X_test = X_test.to(device)
print("정수 인코딩, 패딩, 텐서 변환 완료.")
print(f"최종 테스트 데이터 형태: {X_test.shape}")

# 4. 테스트 데이터 로더 생성
# 테스트용 Dataset은 레이블(y)이 필요 없습니다.
class TestDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx]

test_dataset = TestDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print("테스트 데이터 로더 생성 완료.")

# 5. 모델 예측 수행
# 5.1. 모델을 평가 모드로 전환 (매우 중요!)
model.eval()

# 5.2. 예측 결과를 담을 리스트 생성
test_preds = []

# 5.3. 그래디언트 계산 비활성화
with torch.no_grad():
    for inputs in test_loader:
        # 모델 예측
        outputs = model(inputs)
        
        # 예측값을 0 또는 1로 변환 (sigmoid -> 0.5 기준으로 반올림)
        preds = torch.round(torch.sigmoid(outputs))
        
        # 결과를 리스트에 추가 (CPU로 이동 후 NumPy 배열로 변환)
        test_preds.extend(preds.cpu().numpy().flatten().astype(int))

print("모델 예측 완료.")
print(f"총 예측 개수: {len(test_preds)}")


# 6. 제출 파일(submission.csv) 생성
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_preds
})

submission_path = 'submission_enh.csv'
submission_df.to_csv(submission_path, index=False)

print(f"✅ '{submission_path}' 파일이 성공적으로 생성되었습니다.")
print("제출 파일 샘플:")
print(submission_df.head())

전처리 완료!
컬럼 확인: True
전처리 후 결측치 확인:
id                       0
keyword                  0
location              2533
text                     0
target                   0
tokenized_text           0
non_stopwords_text       0
dtype: int64
유사 단어: [('im', 0.9991193413734436), ('still', 0.9990773797035217), ('fire', 0.9990521669387817), ('us', 0.9990509152412415), ('nuclear', 0.9990345239639282)]
가장 긴 트윗의 길이: 20
Using device: mps
학습 데이터로더 배치 수: 96
검증 데이터로더 배치 수: 24
첫 번째 배치 데이터 형태: torch.Size([64, 20])
첫 번째 배치 레이블 형태: torch.Size([64])
Epoch: 01
	Train Loss: 0.686 | Train Acc: 56.85%
	Val. Loss: 0.683 | Val. Acc: 57.39%
Epoch: 02
	Train Loss: 0.684 | Train Acc: 56.95%
	Val. Loss: 0.682 | Val. Acc: 57.39%
Epoch: 03
	Train Loss: 0.685 | Train Acc: 57.08%
	Val. Loss: 0.683 | Val. Acc: 57.39%
Epoch: 04
	Train Loss: 0.684 | Train Acc: 56.95%
	Val. Loss: 0.682 | Val. Acc: 57.39%
Epoch: 05
	Train Loss: 0.684 | Train Acc: 56.95%
	Val. Loss: 0.682 | Val. Acc: 57.39%
테스트 데이터 로드 완료.
테스트 데이터 전처리 완료 (train